In [1]:
import json

from keras.callbacks import TensorBoard

from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Load data

In [2]:
with open("../data/processed/iris_training_data.json", "r") as f:
    data = json.load(f)

In [3]:
X_train = data.get("X_train")
X_test = data.get("X_test")
Y_train = data.get("Y_train")
Y_test = data.get("Y_test")
n_features = data.get("n_features")
n_classes = data.get("n_classes")

# Create models

In [4]:
def create_model(input_dim, output_dim, nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

models = [create_model(n_features, n_classes, 8, i, f"model_{i}") for i in range(1,4)]

for model in models:
    model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 40        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 27        
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 27        
Total params: 139
Trainable params: 139
Non-tr

# Train

In [5]:
history_dict = {}

cb = TensorBoard()

for model in models:
    print('Model name:', model.name)
    history_callback = model.fit(X_train, Y_train,
                                 batch_size=5,
                                 epochs=50,
                                 verbose=0,
                                 validation_data=(X_test, Y_test),
                                 callbacks=[cb])
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print()
    
    history_dict[model.name] = history_callback.history
    model.save(f"../models/{model.name}")

Model name: model_1
Test loss: 0.2240268737077713
Test accuracy: 0.9333333373069763

Model name: model_2
Test loss: 0.33311203122138977
Test accuracy: 0.8799999952316284

Model name: model_3
Test loss: 0.15216758847236633
Test accuracy: 0.9333333373069763



In [6]:
with open("../data/processed/training_results.json", "w+") as f:
    json.dump(history_dict, f, ensure_ascii=False, indent=4)